# Import libraries

In [ ]:
!pip install ccxt.async_support
!pip install os
!pip install doteny
import ccxt.async_support as ccxt
import os
from dotenv import load_dotenv
import asyncio
from telegram import Bot
import pandas as pd
import math
from telegram import Update
from telegram.ext import Updater, MessageHandler, Filters, CallbackContext
import time 
from datetime import datetime
import os.path
import traceback
from decimal import Decimal
import logging
from decimal import ROUND_DOWN,ROUND_UP
import asyncio
from decimal import Decimal, InvalidOperation
import numpy as np

# Load API keys 

In [ ]:
binance_api_key=YOUR_BINANCE_API_KEY
binance_api_secret=YOUR_BINANCE_API_SECRET

binance_api_key = os.environ.get('binance_api_key')
binance_api_secret = os.environ.get('binance_api_secret')

# Load exchange 

In [ ]:
binance = ccxt.binance({
    'apiKey': binance_api_key,
    'secret': binance_api_secret,
    'enableRateLimit': True
})


# Function for executing trades

In [1]:
#!pip install ccxt

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   ---------------------------------------- 5.7/5.7 MB 43.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 37.7 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 31.2 MB/s eta 0:00:00
Using cached idna-3.10-py3-none-any.whl (70 kB)


In [3]:
# import ccxt
# import time
# from itertools import permutations

# # Initialize Binance API (Public Data Only for Testing)
# exchange = ccxt.binance({
#     'rateLimit': 1200,
#     'enableRateLimit': True,
# })

# # Define the triangular pairs (Example: BTC/USDT, ETH/BTC, ETH/USDT)
# TRIANGULAR_PAIRS = [
#     ('BTC/USDT', 'ETH/BTC', 'ETH/USDT'),
# ]

# # Function to fetch order book data
# def get_order_book(symbol):
#     try:
#         order_book = exchange.fetch_order_book(symbol)
#         return order_book['bids'][0][0], order_book['asks'][0][0]  # Best bid and ask prices
#     except Exception as e:
#         print(f"Error fetching order book for {symbol}: {e}")
#         return None, None

# # Function to calculate arbitrage profit
# def calculate_arbitrage(pair):
#     base_market, inter_market, quote_market = pair
    
#     base_bid, base_ask = get_order_book(base_market)
#     inter_bid, inter_ask = get_order_book(inter_market)
#     quote_bid, quote_ask = get_order_book(quote_market)
    
#     if None in [base_bid, base_ask, inter_bid, inter_ask, quote_bid, quote_ask]:
#         return None
    
#     # Forward Cycle: Start with Quote Currency (USDT)
#     initial_amount = 100  # Testing with 100 USDT
#     step1 = initial_amount / base_ask  # Buy BTC with USDT
#     step2 = step1 / inter_ask  # Buy ETH with BTC
#     final_amount = step2 * quote_bid  # Sell ETH for USDT
    
#     profit_forward = final_amount - initial_amount
    
#     # Reverse Cycle: Start with Base Currency (BTC)
#     step1_rev = initial_amount * inter_bid  # Buy BTC with ETH
#     step2_rev = step1_rev * base_bid  # Sell BTC for USDT
#     profit_reverse = step2_rev - initial_amount
    
#     return profit_forward, profit_reverse

# # Main loop to check arbitrage opportunities
# def run_arbitrage_bot():
#     while True:
#         for pair in TRIANGULAR_PAIRS:
#             profit_forward, profit_reverse = calculate_arbitrage(pair) or (0, 0)
            
#             if profit_forward > 0:
#                 print(f"Arbitrage Opportunity (Forward): {pair} - Profit: {profit_forward:.2f} USDT")
#             if profit_reverse > 0:
#                 print(f"Arbitrage Opportunity (Reverse): {pair} - Profit: {profit_reverse:.2f} USDT")
        
#         time.sleep(2)  # Adjust based on Binance rate limits

# # Run the bot (Testing Mode)
# if __name__ == "__main__":
#     run_arbitrage_bot()


Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 276344.36 USDT
Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 276027.93 USDT
Arbitrage Opportunity (Forward): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 0.03 USDT
Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 276178.31 USDT
Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 276349.00 USDT
Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 276400.90 USDT
Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 276405.09 USDT
Arbitrage Opportunity (Forward): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 0.01 USDT
Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 276347.60 USDT
Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') - Profit: 276256.86 USDT
Arbitrage Opportunity (Reverse): ('BTC/USDT', 'ETH/BTC', 'ETH/USDT') -

KeyboardInterrupt: 

In [ ]:
# live
import ccxt
import time
import csv
from itertools import permutations

# Initialize Binance API (Live Trading Enabled)
exchange = ccxt.binance({
    'apiKey': 'your_api_key',  # Replace with your API key
    'secret': 'your_api_secret',  # Replace with your secret key
    'rateLimit': 1200,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'spot'
    }
})

# Define the triangular pairs (Example: BTC/USDT, ETH/BTC, ETH/USDT)
TRIANGULAR_PAIRS = [
    ('BTC/USDT', 'ETH/BTC', 'ETH/USDT'),
]

# Open CSV file to log data
with open('arbitrage_data.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Timestamp", "Pair", "Profit Forward", "Profit Reverse", "Executed Trades"])

# Function to fetch order book data
def get_order_book(symbol):
    try:
        order_book = exchange.fetch_order_book(symbol)
        return order_book['bids'][0][0], order_book['asks'][0][0]  # Best bid and ask prices
    except Exception as e:
        print(f"Error fetching order book for {symbol}: {e}")
        return None, None

# Function to place an order
def place_order(symbol, side, amount, price):
    try:
        order = exchange.create_limit_order(symbol, side, amount, price)
        return order
    except Exception as e:
        print(f"Error placing order for {symbol}: {e}")
        return None

# Function to execute arbitrage
def execute_arbitrage(pair):
    base_market, inter_market, quote_market = pair
    
    base_bid, base_ask = get_order_book(base_market)
    inter_bid, inter_ask = get_order_book(inter_market)
    quote_bid, quote_ask = get_order_book(quote_market)
    
    if None in [base_bid, base_ask, inter_bid, inter_ask, quote_bid, quote_ask]:
        return
    
    # Forward Cycle: Start with Quote Currency (USDT)
    initial_amount = 100  # Adjust as needed
    step1 = initial_amount / base_ask  # Buy BTC with USDT
    step2 = step1 / inter_ask  # Buy ETH with BTC
    final_amount = step2 * quote_bid  # Sell ETH for USDT
    
    profit_forward = final_amount - initial_amount
    executed_trades = []
    
    if profit_forward > 0:
        print(f"Executing Arbitrage (Forward): {pair} - Profit: {profit_forward:.2f} USDT")
        executed_trades.append(place_order(base_market, 'buy', step1, base_ask))
        executed_trades.append(place_order(inter_market, 'buy', step2, inter_ask))
        executed_trades.append(place_order(quote_market, 'sell', step2, quote_bid))
    
    # Reverse Cycle: Start with Base Currency (BTC)
    step1_rev = initial_amount * inter_bid  # Buy BTC with ETH
    step2_rev = step1_rev * base_bid  # Sell BTC for USDT
    profit_reverse = step2_rev - initial_amount
    
    if profit_reverse > 0:
        print(f"Executing Arbitrage (Reverse): {pair} - Profit: {profit_reverse:.2f} USDT")
        executed_trades.append(place_order(inter_market, 'sell', step1_rev, inter_bid))
        executed_trades.append(place_order(base_market, 'sell', step2_rev, base_bid))
    
    # Log data to CSV
    with open('arbitrage_data.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([time.time(), pair, profit_forward, profit_reverse, len(executed_trades)])

# Main loop to check arbitrage opportunities
def run_arbitrage_bot():
    while True:
        for pair in TRIANGULAR_PAIRS:
            execute_arbitrage(pair)
        
        time.sleep(2)  # Adjust based on Binance rate limits

# Run the bot (Live Trading Mode)
if __name__ == "__main__":
    run_arbitrage_bot()
